# INDEX

- [Initialization](#INITIALIZATION)
    - [Imports](#IMPORTS)
    - [Data Generation](#DATA-GENERATION)
    - [Connection to MongoDB](#CONNECTION-TO-MONGODB)

- [Models](#MODELS)
    - [Model M1](#Model-M1:-Separate-Persons-and-Companies-but-linked-by-references)
    - [Model M2](#Model-M2:-Person-Documents-with-Embedded-Company-Information)
    - [Model M3](#Model-M3:-Company-Documents-with-Embedded-Person-Information)

- [Queries](#QUERIES)
    - [Queries for Model M1](#Queries-and-Times-for-Model-M1)
    - [Queries for Model M2](#Queries-and-Times-for-Model-M2)
    - [Queries for Model M3](#Queries-and-Times-for-Model-M3)

- [Results and Discussion](#RESULTS-AND-DISCUSSION)
    - [Query Execution Times Table](#Query-Execution-Times-per-Model)
    - [Performance Evaluation and Discussion](#Performance-Evaluation-and-Discussion)

# INITIALIZATION

[Back to Index](#INDEX)

## Imports

[Back to Index](#INDEX)

In [181]:
from faker import Faker
import random
import time
from pymongo import MongoClient
import datetime

## Data Generation

[Back to Index](#INDEX)

In [182]:
fake = Faker()
random.seed(98)
Faker.seed(98)

def calculate_age(birthdate, reference_date):
    """ Calculate age based on the birthdate and a reference date """
    return reference_date.year - birthdate.year - ((reference_date.month, reference_date.day) < (birthdate.month, birthdate.day))

def generate_persons(n):
    """ Generate data for n persons with complete details, calculating age from birthdate """
    persons = []
    reference_date = datetime.date(2024, 5, 4)
    for _ in range(n):
        full_name = fake.name()
        first_name = full_name.split()[0]
        birthdate = fake.date_of_birth(minimum_age=18, maximum_age=70)
        age = calculate_age(birthdate, reference_date)
        person = {
            'first_name': first_name,
            'name': full_name,
            'birthdate': birthdate.isoformat(),
            'age': age,
            'email': fake.email(),
            'company_email': first_name.lower() + '@' + fake.domain_name(),
            'sex': fake.random_element(elements=('Male', 'Female', 'Other'))
        }
        persons.append(person)
    return persons

def generate_companies(n):
    """ Generate data for n companies with specified details """
    companies = []
    for _ in range(n):
        company_name = fake.company()
        domain = fake.domain_name()
        company = {
            'name': company_name,
            'email': 'info@' + domain,
            'web_domain': domain,
            'url': 'http://' + domain,
            'vat_number': fake.bothify(text='??#########'),
            'employees': []
        }
        companies.append(company)
    return companies

In [183]:
''' We constrain the number of employees to be between 30 and 100 for each company.'''

def assign_employees_to_companies(persons, companies, min_employees = 30, max_employees = 100):
    """ Assign persons to companies, ensuring each has a realistic number of employees """
    employee_idx = 0
    for company in companies:
        num_employees = random.randint(min_employees, max_employees)
        if employee_idx + num_employees > len(persons):
            num_employees = len(persons) - employee_idx 
        company['employees'].extend(persons[employee_idx:employee_idx + num_employees])
        employee_idx += num_employees
        if employee_idx >= len(persons):
            break

''' We set the model to generate 100k persons and 2k companies in order to ensure having more than 50k documents. '''
num_persons = 100000
num_companies = 2000

persons = generate_persons(num_persons)
companies = generate_companies(num_companies)

assign_employees_to_companies(persons, companies)

# display(persons[0])
# display(companies[0])

## Connect to MongoDB

[Back to Index](#INDEX)

In [184]:
client = MongoClient('localhost', 27017) 

db = client['lab1_database']

## 
[Back to Index](#INDEX)

# MODELS

[Back to Index](#INDEX)

## Model M1: Separate Persons and Companies but linked by references

[Back to Index](#INDEX)

In [185]:
db_m1 = client['lab1_database']

persons_collection = db_m1['persons_m1']
companies_collection = db_m1['companies_m1']

# display(persons_collection.count_documents({}))

# Clean existing data to ensure that we start from scratch
persons_collection.delete_many({})
companies_collection.delete_many({})

# display(persons_collection.count_documents({}))

# Insert M1 data
for company in companies:
    company_data = {
        'name': company['name'],
        'email': company['email'],
        'web_domain': company['web_domain'],
        'url': company['url'],
        'vat_number': company['vat_number']
        }
    company_id = companies_collection.insert_one(company_data).inserted_id
    for employee in company['employees']:
        employee_data = {
            'company_id': company_id,
            'first_name': employee['first_name'],
            'name': employee['name'],
            'birthdate': employee['birthdate'],
            'age': employee['age'],
            'email': employee['email'],
            'company_email': employee['company_email'],
            'sex': employee['sex']
            }
        persons_collection.insert_one(employee_data)

# display(persons_collection.count_documents({}))

## Model M2: Person Documents with Embedded Company Information

[Back to Index](#INDEX)

In [186]:
db_m2 = client['lab1_database']

persons_with_companies_m2 = db_m2['persons_with_companies_m2']

# Clean existing data to ensure that we start from scratch
persons_with_companies_m2.delete_many({})

for company in companies:
    for person in company['employees']:
        person_with_company = {
            'first_name': person['first_name'],
            'name': person['name'],
            'birthdate': person['birthdate'],
            'age': person['age'],
            'email': person['email'],
            'company_email': person['company_email'],
            'sex': person['sex'],
            'company': {
                'name': company['name'],
                'email': company['email'],
                'web_domain': company['web_domain'],
                'url': company['url'],
                'vat_number': company['vat_number']
                }
        }
        persons_with_companies_m2.insert_one(person_with_company)

## Model M3: Company Documents with Embedded Person Information


[Back to Index](#INDEX)

In [187]:
db_m3 = client['lab1_database']

companies_with_persons_m3 = db_m3['companies_with_persons_m3']

# Clean existing data to ensure that we start from scratch
companies_with_persons_m3.delete_many({})

for company in companies:
    company_doc = {
        'name': company['name'],
        'email': company['email'],
        'web_domain': company['web_domain'],
        'url': company['url'],
        'vat_number': company['vat_number'],
        'employees': company['employees']
    }
    companies_with_persons_m3.insert_one(company_doc)

##
[Back to Index](#INDEX)

# QUERIES

[Back to Index](#INDEX)

## Queries and Times for Model M1

[Back to Index](#INDEX)

In [188]:
# Connect to the collection for Model M1
persons_m1 = db['persons_m1']
companies_m1 = db['companies_m1']

In [189]:
# Q1: For each person, retrieve their full name and their company’s name.
persons_results_m1 = []

start_time_q1 = time.time()
for person in persons_m1.find():
    company = companies_m1.find_one({'_id': person['company_id']})
    # print(f"Person: {person['name']}, Company: {company['name']}")
    persons_results_m1.append(f"Person: {person['name']}, Company: {company['name']}")
end_time_q1 = time.time()

for person in persons_results_m1[:10]:
    print(person)

Person: Ronald Gomez, Company: Huang, Horn and Casey
Person: Renee Wolf, Company: Huang, Horn and Casey
Person: Kimberly Powell, Company: Huang, Horn and Casey
Person: Stanley Ingram, Company: Huang, Horn and Casey
Person: Kelsey Watson, Company: Huang, Horn and Casey
Person: Ryan Owens, Company: Huang, Horn and Casey
Person: Matthew Smith, Company: Huang, Horn and Casey
Person: Yvonne Jensen, Company: Huang, Horn and Casey
Person: Paul Jenkins, Company: Huang, Horn and Casey
Person: Patrick Howard, Company: Huang, Horn and Casey


In [190]:
# Q2: For each company, retrieve its name and the number of employees.
company_results_m1 = []

start_time_q2 = time.time()
for company in companies_m1.find():
    employee_count = persons_m1.count_documents({'company_id': company['_id']})
    # print(f"Company: {company['name']}, Number of Employees: {employee_count}")
    company_results_m1.append(f"Company: {company['name']}, Number of Employees: {employee_count}")
end_time_q2 = time.time()

for company in company_results_m1[:10]:
    print(company)

Company: Huang, Horn and Casey, Number of Employees: 75
Company: Moran PLC, Number of Employees: 35
Company: Stuart LLC, Number of Employees: 75
Company: Garcia-Williams, Number of Employees: 83
Company: Kennedy-Harvey, Number of Employees: 37
Company: Lewis, Holland and Bailey, Number of Employees: 90
Company: Sharp, Gibson and Lewis, Number of Employees: 66
Company: Harris-Jenkins, Number of Employees: 38
Company: Schneider Inc, Number of Employees: 93
Company: Thompson-Peters, Number of Employees: 55


In [191]:
# Q3: For each person born before 1988, update their age to “30”.
start_time_q3 = time.time()
persons_m1.update_many({'birthdate': {'$lt': '1988-01-01'}}, {'$set': {'age': 30}})
end_time_q3 = time.time()

for person in persons_m1.find({'birthdate': {'$gte': '1987-01-01'}}).limit(10):
    print(person)

{'_id': ObjectId('6637f960264ae30efff83ac3'), 'company_id': ObjectId('6637f960264ae30efff83abf'), 'first_name': 'Stanley', 'name': 'Stanley Ingram', 'birthdate': '1997-02-24', 'age': 27, 'email': 'marieedwards@example.org', 'company_email': 'stanley@phillips.com', 'sex': 'Female'}
{'_id': ObjectId('6637f960264ae30efff83ac4'), 'company_id': ObjectId('6637f960264ae30efff83abf'), 'first_name': 'Kelsey', 'name': 'Kelsey Watson', 'birthdate': '1988-07-11', 'age': 35, 'email': 'danielsmonica@example.org', 'company_email': 'kelsey@turner-lewis.com', 'sex': 'Male'}
{'_id': ObjectId('6637f960264ae30efff83ac8'), 'company_id': ObjectId('6637f960264ae30efff83abf'), 'first_name': 'Paul', 'name': 'Paul Jenkins', 'birthdate': '1997-11-08', 'age': 26, 'email': 'creyes@example.net', 'company_email': 'paul@johnson.com', 'sex': 'Other'}
{'_id': ObjectId('6637f960264ae30efff83acc'), 'company_id': ObjectId('6637f960264ae30efff83abf'), 'first_name': 'Michele', 'name': 'Michele Schmidt', 'birthdate': '2005-0

In [192]:
# Q4: For each company, update its name to include the word “Company”.
start_time_q4 = time.time()
companies_m1.update_many({}, [{'$set': {'name': {'$concat': ['$name', ' Company']}}}])
end_time_q4 = time.time()

for company in companies_m1.find().limit(10):
    print(company['name'])

Huang, Horn and Casey Company
Moran PLC Company
Stuart LLC Company
Garcia-Williams Company
Kennedy-Harvey Company
Lewis, Holland and Bailey Company
Sharp, Gibson and Lewis Company
Harris-Jenkins Company
Schneider Inc Company
Thompson-Peters Company


In [193]:
print(f"Query Q1 execution time (M1): {end_time_q1 - start_time_q1} seconds")
print(f"Query Q2 execution time (M1): {end_time_q2 - start_time_q2} seconds")
print(f"Query Q3 execution time (M1): {end_time_q3 - start_time_q3} seconds")
print(f"Query Q4 execution time (M1): {end_time_q4 - start_time_q4} seconds")

Query Q1 execution time (M1): 10.417908906936646 seconds
Query Q2 execution time (M1): 25.194710969924927 seconds
Query Q3 execution time (M1): 0.37863683700561523 seconds
Query Q4 execution time (M1): 0.01661396026611328 seconds


## Queries and Times for Model M2

[Back to Index](#INDEX)

In [194]:
# Connect to the collection for Model M2
persons_with_companies_m2 = db['persons_with_companies_m2']

In [195]:
# Q1: For each person, retrieve their full name and their company’s name.
persons_results_m2 = []

start_time_q1 = time.time()
for person in persons_with_companies_m2.find():
    # print(f"Person: {person['name']}, Company: {person['company']['name']}")
    persons_results_m2.append(f"Person: {person['name']}, Company: {person['company']['name']}")
end_time_q1 = time.time()

for person in persons_results_m2[:10]:
    print(person)

Person: Ronald Gomez, Company: Huang, Horn and Casey
Person: Renee Wolf, Company: Huang, Horn and Casey
Person: Kimberly Powell, Company: Huang, Horn and Casey
Person: Stanley Ingram, Company: Huang, Horn and Casey
Person: Kelsey Watson, Company: Huang, Horn and Casey
Person: Ryan Owens, Company: Huang, Horn and Casey
Person: Matthew Smith, Company: Huang, Horn and Casey
Person: Yvonne Jensen, Company: Huang, Horn and Casey
Person: Paul Jenkins, Company: Huang, Horn and Casey
Person: Patrick Howard, Company: Huang, Horn and Casey


In [196]:
# Q2: For each company, retrieve its name and the number of employees.
company_results_m2 = []

start_time_q2 = time.time()
pipeline = [
    {"$group": {"_id": "$company.name", "num_employees": {"$sum": 1}}}
]
results = persons_with_companies_m2.aggregate(pipeline)
for result in results:
    # print(f"Company: {result['_id']}, Number of Employees: {result['num_employees']}")
    company_results_m2.append(f"Company: {result['_id']}, Number of Employees: {result['num_employees']}")
end_time_q2 = time.time()

for company in company_results_m2[:10]:
    print(company)

Company: Schaefer-Moore, Number of Employees: 100
Company: Carroll Inc, Number of Employees: 55
Company: Fields, Young and Wilson, Number of Employees: 83
Company: Allen-Reid, Number of Employees: 82
Company: Fuller-Nichols, Number of Employees: 64
Company: Stewart PLC, Number of Employees: 77
Company: Blankenship Ltd, Number of Employees: 92
Company: Weber, Jenkins and Taylor, Number of Employees: 45
Company: Lester LLC, Number of Employees: 90
Company: Sparks Inc, Number of Employees: 72


In [197]:
# Q3: For each person born before 1988, update their age to “30”.
start_time_q3 = time.time()
persons_with_companies_m2.update_many(
    {"birthdate": {"$lt": "1988-01-01"}},
    {"$set": {"age": 30}}
)
end_time_q3 = time.time()

for person in persons_with_companies_m2.find({'birthdate': {'$gte': '1987-01-01'}}).limit(10):
    print(person)

{'_id': ObjectId('6637f969264ae30efff9c932'), 'first_name': 'Stanley', 'name': 'Stanley Ingram', 'birthdate': '1997-02-24', 'age': 27, 'email': 'marieedwards@example.org', 'company_email': 'stanley@phillips.com', 'sex': 'Female', 'company': {'name': 'Huang, Horn and Casey', 'email': 'info@goodwin-wilson.com', 'web_domain': 'goodwin-wilson.com', 'url': 'http://goodwin-wilson.com', 'vat_number': 'xE025410621'}}
{'_id': ObjectId('6637f969264ae30efff9c933'), 'first_name': 'Kelsey', 'name': 'Kelsey Watson', 'birthdate': '1988-07-11', 'age': 35, 'email': 'danielsmonica@example.org', 'company_email': 'kelsey@turner-lewis.com', 'sex': 'Male', 'company': {'name': 'Huang, Horn and Casey', 'email': 'info@goodwin-wilson.com', 'web_domain': 'goodwin-wilson.com', 'url': 'http://goodwin-wilson.com', 'vat_number': 'xE025410621'}}
{'_id': ObjectId('6637f969264ae30efff9c937'), 'first_name': 'Paul', 'name': 'Paul Jenkins', 'birthdate': '1997-11-08', 'age': 26, 'email': 'creyes@example.net', 'company_emai

In [198]:
# Q4: For each company embedded in persons, update its name to include the word “Company”.
start_time_q4 = time.time()
persons_with_companies_m2.update_many(
    {},
    [{'$set': {'company.name': {'$concat': ['$company.name', ' Company']}}}]
)
end_time_q4 = time.time()

for person in persons_with_companies_m2.find().limit(10):
    print(person['company']['name'])

Huang, Horn and Casey Company
Huang, Horn and Casey Company
Huang, Horn and Casey Company
Huang, Horn and Casey Company
Huang, Horn and Casey Company
Huang, Horn and Casey Company
Huang, Horn and Casey Company
Huang, Horn and Casey Company
Huang, Horn and Casey Company
Huang, Horn and Casey Company


In [199]:
print(f"Query Q1 execution time (M2): {end_time_q1 - start_time_q1} seconds")
print(f"Query Q2 execution time (M2): {end_time_q2 - start_time_q2} seconds")
print(f"Query Q3 execution time (M2): {end_time_q3 - start_time_q3} seconds")
print(f"Query Q4 execution time (M2): {end_time_q4 - start_time_q4} seconds")

Query Q1 execution time (M2): 0.4828310012817383 seconds
Query Q2 execution time (M2): 0.062248945236206055 seconds
Query Q3 execution time (M2): 0.3370811939239502 seconds
Query Q4 execution time (M2): 0.9005098342895508 seconds


## Queries and Times for Model M3

[Back to Index](#INDEX)

In [200]:
# Connect to the collection for Model M3
companies_with_persons_m3 = db['companies_with_persons_m3']

In [201]:
# Q1: For each person, retrieve their full name and their company’s name.
persons_results_m3 = []

start_time_q1 = time.time()
for company in companies_with_persons_m3.find():
    company_name = company['name']
    for person in company['employees']:
        # print(f"Person: {person['name']}, Company: {company_name}")
        persons_results_m3.append(f"Person: {person['name']}, Company: {company_name}")
end_time_q1 = time.time()

for person in persons_results_m3[:10]:
    print(person)

Person: Ronald Gomez, Company: Huang, Horn and Casey
Person: Renee Wolf, Company: Huang, Horn and Casey
Person: Kimberly Powell, Company: Huang, Horn and Casey
Person: Stanley Ingram, Company: Huang, Horn and Casey
Person: Kelsey Watson, Company: Huang, Horn and Casey
Person: Ryan Owens, Company: Huang, Horn and Casey
Person: Matthew Smith, Company: Huang, Horn and Casey
Person: Yvonne Jensen, Company: Huang, Horn and Casey
Person: Paul Jenkins, Company: Huang, Horn and Casey
Person: Patrick Howard, Company: Huang, Horn and Casey


In [202]:
# Q2: For each company, retrieve its name and the number of employees.
company_results_m3 = []

start_time_q2 = time.time()
for company in companies_with_persons_m3.find():
    employee_count = len(company['employees'])
    # print(f"Company: {company['name']}, Number of Employees: {employee_count}")
    company_results_m3.append(f"Company: {company['name']}, Number of Employees: {employee_count}")
end_time_q2 = time.time()

for company in company_results_m3[:10]:
    print(company)

Company: Huang, Horn and Casey, Number of Employees: 75
Company: Moran PLC, Number of Employees: 35
Company: Stuart LLC, Number of Employees: 75
Company: Garcia-Williams, Number of Employees: 83
Company: Kennedy-Harvey, Number of Employees: 37
Company: Lewis, Holland and Bailey, Number of Employees: 90
Company: Sharp, Gibson and Lewis, Number of Employees: 66
Company: Harris-Jenkins, Number of Employees: 38
Company: Schneider Inc, Number of Employees: 93
Company: Thompson-Peters, Number of Employees: 55


In [203]:
# Q3: For each person in each company born before 1988, update their age to “30”.
start_time_q3 = time.time()
companies_with_persons_m3.update_many(
    {"employees.birthdate": {"$lt": "1988-01-01"}},
    {"$set": {"employees.$[elem].age": 30}},
    array_filters=[{"elem.birthdate": {"$lt": "1988-01-01"}}]
)
end_time_q3 = time.time()

pipeline = [
    {"$unwind": "$employees"}, 
    {"$match": {"employees.birthdate": {"$gte": "1987-01-01"}}},
    {"$limit": 10}
]
results = companies_with_persons_m3.aggregate(pipeline)

for person in results:
    print(person) 

{'_id': ObjectId('6637f972264ae30efffb4fcf'), 'name': 'Huang, Horn and Casey', 'email': 'info@goodwin-wilson.com', 'web_domain': 'goodwin-wilson.com', 'url': 'http://goodwin-wilson.com', 'vat_number': 'xE025410621', 'employees': {'first_name': 'Stanley', 'name': 'Stanley Ingram', 'birthdate': '1997-02-24', 'age': 27, 'email': 'marieedwards@example.org', 'company_email': 'stanley@phillips.com', 'sex': 'Female'}}
{'_id': ObjectId('6637f972264ae30efffb4fcf'), 'name': 'Huang, Horn and Casey', 'email': 'info@goodwin-wilson.com', 'web_domain': 'goodwin-wilson.com', 'url': 'http://goodwin-wilson.com', 'vat_number': 'xE025410621', 'employees': {'first_name': 'Kelsey', 'name': 'Kelsey Watson', 'birthdate': '1988-07-11', 'age': 35, 'email': 'danielsmonica@example.org', 'company_email': 'kelsey@turner-lewis.com', 'sex': 'Male'}}
{'_id': ObjectId('6637f972264ae30efffb4fcf'), 'name': 'Huang, Horn and Casey', 'email': 'info@goodwin-wilson.com', 'web_domain': 'goodwin-wilson.com', 'url': 'http://good

In [204]:
# Q4: Update each company's name to include the word “Company”.
start_time_q4 = time.time()
companies_with_persons_m3.update_many({}, [{'$set': {'name': {'$concat': ['$name', ' Company']}}}])
end_time_q4 = time.time()

for company in companies_with_persons_m3.find().limit(10):
    print(company['name'])

Huang, Horn and Casey Company
Moran PLC Company
Stuart LLC Company
Garcia-Williams Company
Kennedy-Harvey Company
Lewis, Holland and Bailey Company
Sharp, Gibson and Lewis Company
Harris-Jenkins Company
Schneider Inc Company
Thompson-Peters Company


In [205]:
print(f"Query Q1 execution time (M3): {end_time_q1 - start_time_q1} seconds")
print(f"Query Q2 execution time (M3): {end_time_q2 - start_time_q2} seconds")
print(f"Query Q3 execution time (M3): {end_time_q3 - start_time_q3} seconds")
print(f"Query Q4 execution time (M3): {end_time_q4 - start_time_q4} seconds")

Query Q1 execution time (M3): 0.18879914283752441 seconds
Query Q2 execution time (M3): 0.11036205291748047 seconds
Query Q3 execution time (M3): 0.09303092956542969 seconds
Query Q4 execution time (M3): 0.18846511840820312 seconds


##
[Back to Index](#INDEX)

# RESULTS AND DISCUSSION

[Back to Index](#INDEX)

## Query Execution Times per Model

[Back to Index](#INDEX)

<div align="center">
<table>
    <thead>
        <tr>
            <th></th>
            <th>Q1</th>
            <th>Q2</th>
            <th>Q3</th>
            <th>Q4</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><strong>M1</strong></td>
            <td> 11.0651 seconds </td>
            <td> 26.3386 seconds </td>
            <td> 0.4349 seconds </td>
            <td> 0.0466 seconds </td>
        </tr>
        <tr>
            <td><strong>M2</strong></td>
            <td> 0.5441 seconds </td>
            <td> 0.0938 seconds </td>
            <td> 0.3366 seconds </td>
            <td> 0.9650 seconds </td>
        </tr>
        <tr>
            <td><strong>M3</strong></td>
            <td> 0.1644 seconds </td>
            <td> 0.1090 seconds </td>
            <td> 0.1397 seconds </td>
            <td> 0.1477 seconds </td>
        </tr>
    </tbody>
</table>
</div>

## Performance Evaluation and Discussion

[Back to Index](#INDEX)

**1. Order queries from best to worst for Q1. Which model performs best? Why?**

- Best: M3
- Second Best: M2
- Worst: M1

The Best Model in this case is M3. We can observe a very big gap between M1, which has completely separated documents and, thus, takes a very long time to get the company related to each employee, the the models with embedded data in a single collection. From those, M3 gains an edge because it has to get the company name only once for all the employees in the company. M2, on the other hand, has to get the company name for each employee, which takes some additional time, although not as much more as M1.

**2. Order queries from best to worst for Q2. Which model performs best? Why?**

- Best: M2
- Second Best: M3
- Worst: M1

The Best Model in this case is M2. Again, we can observe a very big gap between M1, which has completely separated documents and, thus, takes a very long time to get the relationship for each employee. The gap is even bigger in this case because, for each company, we have to search through all the employes in order to find those who work in it and then count them. M3 and M2, on the other hand, already have this information embedded in each document that we access. M2 is only slightly better than M3 but the reason is not clear entirely. In M2 we group persons by company and we can aggregate them in a very efficient manner while in M3 we have to count the length of the "employees" array for each company, which probably is a bit slower. The difference would only be significant for very large datasets, as even with a dataset of more than 50k documents the difference is less than 2 centiseconds.

**3. Order queries from worst to best for Q3. Which model performs worst? Why?**

- Best: M3
- Second Best: M2
- Worst: M1

The Best Model in this case is M3. The differences in this case are the narrowest out of the 4 queries, but nevertheless the difference between M3 and M1 or M2 can become signficant decently quickly. In this case, both M1 and M2 have to iterate through all the persons documents in order to get to their birthdays and then filter to update age to 30. In this case, M2 could be slightly faster because in M1 there is one additional information section ("company_id") before the birthday and age. M3, on the other hand, goes first through the filter so it can already avoid having to iterate through all the documents which don't have employees born before 1988. Additionally, M3 also has between 30 and 100 employees in each document because their personal information is embedded within each company's file. This means it has to enter less files and, thus, saves time on that. As a result, M3 optimizes the updating process a little bit more than M2 and M1, which don't fall as much behind as M1 did in previous queries, but still do perform decidedly worse.

**4. Order queries from worst to best for Q4. Which model performs worst? Why?**

- Best: M1
- Second Best: M3
- Worst: M2

The Best Model in this case is M1. This is the only query where M1 performs better than M2 and M3. The reason is very simple, in this query we update every company's name, so with M1 we minimize the amount of files to access (we only access each company once, as we do as well in M3), but we also minimize the size of each of those files, which speeds up the process a little bit more. M3 also has decent performance because we only have to access one document per company, but the size of each document we have to open is significantly bigger because it contains all the information about all the employees for the company. The worst performance comes from M2 because in this case, to update the company names we have to access the name through each employee, which significantly increases the amount of files we have to open and the amount of times that we have to perform the action of updating the name. For a company of 100 employees in M2 we would be accessing 100 documents for each company while for M1 and M3 we would only access one document.

**5. What are your conclusions about denormalization or normalization of data in MongoDB? In the case of updates, which offers better performance?**

Usually normalization might be better to avoid redundancy, improve data integrity and decrease the size of the dataset but, as we have seen, this comes at the cost that most queries will run slower, some times even MUCH slower. As a result, the choice to normalize the data is far from incidental and should be made with careful consideration of the types of queries that will be usually run. If the most usual type of queries to be run include retrieving information that can only be accessed through referencing one collection to another, then the improvement in data integrity and size has to be really important in order to justify the huge performance loss. On the other hand, if the usual queries only need to retrieve or manipulate information from one collection at a time, then the performance loss might very quickly be compensated otherwise. 

Additionally, the choice of exactly how to store the data in a non-normalized way or to denormalize it is also not entirely banal, but its repercussions are far less relevant than those of the decision to normalize or not. With one non-normalized system or the other the difference in performance might even be negligible while normalizing can potentially lead to runtimes that are more than 200 times slower.

As a final touch, normalization can be a good choice for performance reasons as we saw for query 4, but the circumstances in which this might be the case are very specific. As a result, we would run very quickly into the danger of needing other types of queries which step out of those circumstances and run much slower so the performance gained  by normalization might be negatively compensated by the performance loss in those other queries.

##
[Back to Index](#INDEX)

#
[Back to Index](#INDEX)